<a href="https://colab.research.google.com/github/nhanguyene/MIDTERM_TEST_AI_NGUYEN_TRONG_NHAN/blob/main/CNN_Detect_Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#Import libraries
import glob
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from random import randint
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [25]:
#Connect with my drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
#Get dataset from drive
train=ImageDataGenerator(rescale=1/255)
validation=ImageDataGenerator(rescale=1/255)
training_set=train.flow_from_directory('/content/drive/MyDrive/AI/NEW/Emotion/train/train',target_size=(150,150), batch_size=40, class_mode='categorical')
validation_set=validation.flow_from_directory('/content/drive/MyDrive/AI/NEW/Emotion/train/val',target_size=(150,150), batch_size=40, class_mode='categorical')

Found 60 images belonging to 4 classes.
Found 48 images belonging to 4 classes.


In [27]:
#Check label of dataset
training_set.class_indices

{'Disgust': 0, 'Fear': 1, 'Happy': 2, 'Surprise': 3}

In [28]:
#Create Model for train
model = Sequential()
model.add(Conv2D(16,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(10,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 150, 150, 16)      448       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 75, 75, 16)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 75, 75, 32)        4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 37, 37, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 37, 37, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 18, 18, 64)      

In [ ]:
#Training
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=RMSprop(),loss='categorical_crossentropy',metrics=['accuracy'])
from keras.callbacks import EarlyStopping
history = model.fit(training_set, epochs = 10, validation_data = validation_set, verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

In [ ]:
#Save a file after train
model.save('CNN_EMOTION.h5')

In [ ]:
#Draw plot and evaluate
score = model.evaluate(validation_set,verbose=0)
print('Test error: ',score[0])
print('Test accuracy: ',score[1])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train','Validation'])
plt.show()

In [ ]:
#Load trained file
CNN_FRUIT =load_model('CNN_EMOTION.h5')

In [ ]:
#Check detect
img_path = '/content/drive/MyDrive/AI/NEW/Emotion/test/Fear/fear_face_31.jpeg'
img=load_img(img_path,target_size=(150,150))
plt.imshow(img)
img=img_to_array(img)
img=img.reshape(1,150,150,3)
img=img.astype('float32')
img=img/255
Emotion=np.argmax(CNN_EMOTION.predict(img),axis=1)
pred = model.predict(img)
classes = ['Disgust','Fear','Happy','Surprise','plums']
print(np.argmax(pred))
if Emotion==0: print(classes[0])
elif Emotion==1: print(classes[1])
elif Emotion==2: print(classes[2])
elif Emotion==3: print(classes[3])